In [38]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestClassifier
from IPython.display import display
from sklearn import metrics

In [3]:
df_train = pd.read_feather(r'tmp\train_v6')
df_test = pd.read_feather(r'tmp\test_v6')

In [4]:
y_df = df_train['is_promoted']

In [5]:
df_train = df_train.drop(columns=['is_promoted','employee_id'], axis=1)

In [6]:
len(df_train.columns), df_train.columns

(24,
 Index(['no_of_trainings', 'length_of_service', 'KPIs_met >80%', 'awards_won?',
        'avg_training_score', 'department_Analytics', 'department_Finance',
        'department_HR', 'department_Legal', 'department_Operations',
        'department_Procurement', 'department_R&D',
        'department_Sales & Marketing', 'department_Technology',
        'education_Bachelor's', 'education_Below Secondary',
        'education_Master's & above', 'education_Missing',
        'previous_year_rating_0.0', 'previous_year_rating_1.0',
        'previous_year_rating_2.0', 'previous_year_rating_3.0',
        'previous_year_rating_4.0', 'previous_year_rating_5.0'],
       dtype='object'))

In [7]:
X = df_train.iloc[:,:].values

In [8]:
X.shape

(54808, 24)

In [9]:
y = y_df.iloc[:].values

In [10]:
y.shape

(54808,)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [14]:
X_train = X.copy()
y_train = y.copy()

In [ ]:
(np.unique(y_valid, return_counts=True), np.unique(y_train, return_counts=True))

Training Different Models and evaluating f1 score.

In [15]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier as xgb

In [22]:
#parameters = [{'learning_rate': [ 0.2,0.25, 0.3], 'n_estimators' : [100, 120, 140, 160]}]
#classifier = xgb(random_state=42, n_jobs=4)
#grid_search = GridSearchCV(estimator = classifier, param_grid = parameters,scoring = 'f1',cv = 10, n_jobs = 4)
#grid_search = grid_search.fit(X_train, y_train)
#best_accuracy = grid_search.best_score_
#best_parameters = grid_search.best_params_
#best_accuracy, best_parameters

XG Boost model gave the F1 score of 0.51185 on validation set.

In [18]:
xgb_classifier = xgb(random_state=42, n_jobs=4, learning_rate=0.3, n_estimators=140, max_depth=6, min_child_weight=6, 
                 subsample=1, colsample_bytree=1, colsample_bylevel=1)
xgb_classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.3, max_delta_step=0,
       max_depth=6, min_child_weight=6, missing=None, n_estimators=140,
       n_jobs=4, nthread=None, objective='binary:logistic',
       random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [ ]:
#Working XG Boost Model
#xgb_classifier = xgb(random_state=42, n_jobs=4, learning_rate=0.3, n_estimators=140, max_depth=6, min_child_weight=6, 
#                 subsample=1, colsample_bytree=1, colsample_bylevel=1)
#xgb_classifier.fit(X_train, y_train)
y_xgb_prob = xgb_classifier.predict_proba(X_valid)
#print(metrics.f1_score(y_valid, y_hat))
#print(metrics.confusion_matrix(y_valid, y_hat))

SVC gave 0.4980 F1 score on validation set

In [19]:
#Working Support vector classifier.
classifier = SVC(random_state=42, C=1000, gamma=0.2, kernel='rbf', probability=True )
classifier.fit(X_train, y_train)

SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.2, kernel='rbf',
  max_iter=-1, probability=True, random_state=42, shrinking=True,
  tol=0.001, verbose=False)

In [19]:
#Working Support vector classifier.
#classifier = SVC(random_state=42, C=1000, gamma=0.2, kernel='rbf' )
#classifier.fit(X_train, y_train)
y_svc_prob = classifier.predict_proba(X_valid)
#print(metrics.f1_score(y_valid, y_hat))
#print(metrics.confusion_matrix(y_valid, y_hat))

In [86]:
# m = RandomForestClassifier(n_estimators=40, min_samples_leaf=3, n_jobs=-1, max_features=0.6, random_state=42)
# m.fit(X_train, y_train)
# y_hat = m.predict(X_valid)
# metrics.f1_score(y_valid, y_hat)

0.5085470085470085

In [20]:
m = RandomForestClassifier(n_estimators=40, min_samples_leaf=3, n_jobs=-1, max_features=0.6, random_state=42, 
                                    criterion='entropy', max_depth=40)
m.fit(X_train, y_train)
#y_rf_prob = m.predict_proba(X_valid)
#metrics.f1_score(y_valid, y_hat)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=40, max_features=0.6, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [21]:
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator

In [23]:
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=GradientBoostingClassifier(learning_rate=0.5, max_depth=4, max_features=0.55, min_samples_leaf=18, min_samples_split=11, n_estimators=100, subsample=0.8500000000000001)),
    ExtraTreesClassifier(bootstrap=False, criterion="gini", max_features=0.7000000000000001, min_samples_leaf=16, min_samples_split=16, n_estimators=100)
)
exported_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('stackingestimator', StackingEstimator(estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.5, loss='deviance', max_depth=4,
              max_features=0.55, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=No...imators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False))])

In [23]:
y_tpot_prob = exported_pipeline.predict_proba(X_valid)

In [36]:
all_preds = np.stack((y_xgb_prob, y_svc_prob, y_rf_prob, y_tpot_prob))

In [37]:
all_preds.shape

(4, 16443, 2)

In [38]:
avg_preds = all_preds.mean(axis=0)

In [39]:
avg_preds.shape

(16443, 2)

In [47]:
avg_preds_1 = avg_preds[:, 1]

In [50]:
avg_preds_1.shape

(16443,)

In [55]:
avg_preds_1[avg_preds_1>0.5].shape, avg_preds_1[avg_preds_1>0.3].shape, avg_preds_1[avg_preds_1>0.35].shape

((507,), (719,), (612,))

In [62]:
n = np.array(avg_preds_1)
n[np.abs(n) < 0.4] = 0
n[np.abs(n) >= 0.4] = 1
y_hat = n

In [63]:
print (metrics.f1_score(y_valid, y_hat))
print(metrics.confusion_matrix(y_valid, y_hat))

0.5147679324894514
[[15035    75]
 [  845   488]]


In [16]:
import tpot

In [ ]:
from tpot import TPOTClassifier
tpot = TPOTClassifier(generations=10, population_size=20, verbosity=2, scoring='f1', n_jobs=4, early_stop=10, random_state=42)
tpot.fit(X_train, y_train)

In [ ]:
# to be executed after topt is run successfully and save this model.
print(tpot.score(X_test, y_test))
tpot.export('tpot-mnist-pipeline_30.py')
tpot.predict(X_test)

# Preparing for submission.

In [24]:
df_test.drop(columns='employee_id', axis=1, inplace=True)

In [25]:
X_test = df_test.iloc[:,:].values

In [26]:
y_test_xgf = xgb_classifier.predict_proba(X_test)
y_test_svc = classifier.predict_proba(X_test)
y_test_rf = m.predict_proba(X_test)
y_test_tpot = exported_pipeline.predict_proba(X_test)

In [27]:
all_preds = np.stack((y_test_xgf, y_test_svc, y_test_rf, y_test_tpot))
print(all_preds.shape)
avg_preds = all_preds.mean(axis=0)
print(avg_preds.shape)
avg_preds_1 = avg_preds[:, 1]
print(avg_preds_1.shape)

(4, 23490, 2)
(23490, 2)
(23490,)


In [28]:
n = np.array(avg_preds_1)
n[np.abs(n) < 0.4] = 0
n[np.abs(n) >= 0.4] = 1
y_test = n

In [56]:
#y_test = m.predict(X_test)

In [29]:
load_test_csv = pd.read_csv(r'C:\Users\GJ\Desktop\Machine Learning\fastai\WNS\test.csv', low_memory=False)

In [31]:
load_test_csv.insert(13, 'is_promoted', y_test)

In [32]:
np.unique(y_test, return_counts=True)

(array([0., 1.]), array([22696,   794], dtype=int64))

In [33]:
load_test_csv.drop(columns=['department', 'region'], inplace=True)

In [34]:
load_test_csv.drop(columns=['education', 'gender', 'recruitment_channel','no_of_trainings','age','previous_year_rating' ], inplace=True)

In [35]:
load_test_csv.drop(columns=['length_of_service', 'KPIs_met >80%', 'awards_won?', 'avg_training_score'], inplace=True)

In [36]:
load_test_csv.head()

,employee_id,is_promoted
0,8724,0.0
1,74430,0.0
2,72255,0.0
3,38562,0.0
4,64486,0.0


In [37]:
load_test_csv.to_csv(r'tmp/results_v3_1.csv', index=False)